In [3]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

# MNIST 데이터 적재
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

os.environ['TF_CPP_MIN_LOG_LEVEL']='2' # 경고 메시지 화면출력 금지

#파라미터
learning_rate = 0.001 #신경망 학습률
training_epochs = 100 # 학습 횟수 (epoch)
batch_size = 100 # 미니배치의 크기
display_step = 10 # 중간결과 출력 간격

#신경망 구조 관련 파라미터
n_hidden_1 = 15 # 은닉층의 노드 개수
n_input = 784 #입력층의 노드 개수 MNIST 데이터 (28x28)
n_classes = 10 # 출력층의 노드 수 MNIST 부류 개수(숫자 0~9)

#텐서 그래프 입력 변수
x = tf.placeholder("float", [None, n_input]) #입력 : 필기체 영상
y = tf.placeholder("float", [None, n_classes]) #출력 : 숫자

#학습모델 MLP정의
def multilayer_perceptron(x, weights, biases):
    #ReLu를 사용하는 은닉층
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    #출력층 (활성화 함수 미사용)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

#학습할 파라미터: 가중치(weights), 편차항(biases)
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

#신경망 모델 구성, 출력값 pred : 입력 x에 대한 신경망의 출력
pred = multilayer_perceptron(x, weights, biases)

with tf.name_scope("cost"):
    #비용(오차) 정의 (신경망 출력 pred, 목표 출력 y): 교차 엔트로피 사용
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    #학습 알고리즘 설정
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    tf.summary.scalar('cost', cost)

with tf.name_scope("accuracy"):
    #모델 테스트 : out의 최대값 노드와 y 노드가 같으면 정답
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    #correct_prediction 평균
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    tf.summary.scalar('accuracy', accuracy)

    
init = tf.global_variables_initializer() #변수 초기화 지정

#데이터 플로우 그래프 실행
with tf.Session() as sess:
    print("Start....")
    writer = tf.summary.FileWriter("./logs/fucking_logs", sess.graph)  # for 0.8
    merged = tf.summary.merge_all()
    
    sess.run(init)
    summary, acc = sess.run([merged, accuracy], feed_dict={x: mnist.test.images, y: mnist.test.labels})
    total_batch = int(mnist.train.num_examples / batch_size) #배치 개수

    for epoch in range(training_epochs): #정해진 횟수 만큼 학습
        avg_cost = 0.
        for i in range(total_batch): #미니 배치
            batch_x, batch_y = mnist.train.next_batch(batch_size) #적재
            #역전파 알고리즘 적용
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch #평균 손실(오류) 계산

            if epoch %display_step == 0: #현재 학습 상황 출력
                print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            summary, acc = sess.run([merged, accuracy], feed_dict={x: mnist.test.images, y: mnist.test.labels})
            writer.add_summary(summary, i)
    
    print(accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    print("End...")
    


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Start....
Epoch: 0001 cost= 0.058835699
Epoch: 0001 cost= 0.114240074
Epoch: 0001 cost= 0.161470802
Epoch: 0001 cost= 0.214088780
Epoch: 0001 cost= 0.259045105
Epoch: 0001 cost= 0.307635144
Epoch: 0001 cost= 0.358324512
Epoch: 0001 cost= 0.405354573
Epoch: 0001 cost= 0.448498816
Epoch: 0001 cost= 0.496896879
Epoch: 0001 cost= 0.544603927
Epoch: 0001 cost= 0.589191551
Epoch: 0001 cost= 0.629140438
Epoch: 0001 cost= 0.668647007
Epoch: 0001 cost= 0.712686379
Epoch: 0001 cost= 0.753863061
Epoch: 0001 cost= 0.796631220
Epoch: 0001 cost= 0.839634028
Epoch: 0001 cost= 0.879658872
Epoch: 0001 cost= 0.919996279
Epoch: 0001 cost= 0.965266321
Epoch: 0001 cost= 1.009598677
Epoch: 0001 cost= 1.052000160
Epoch: 0001 cost= 1.090414276
Epoch: 0001 cost= 1.129007450
Epoch: 0001 cost= 1.168607382
Epoch: 0001 cost

Epoch: 0001 cost= 5.653119097
Epoch: 0001 cost= 5.661344635
Epoch: 0001 cost= 5.668843689
Epoch: 0001 cost= 5.677303435
Epoch: 0001 cost= 5.684769221
Epoch: 0001 cost= 5.693765867
Epoch: 0001 cost= 5.701697377
Epoch: 0001 cost= 5.707737748
Epoch: 0001 cost= 5.714945970
Epoch: 0001 cost= 5.722852079
Epoch: 0001 cost= 5.730131697
Epoch: 0001 cost= 5.738630784
Epoch: 0001 cost= 5.745948103
Epoch: 0001 cost= 5.753710909
Epoch: 0001 cost= 5.761606628
Epoch: 0001 cost= 5.767409995
Epoch: 0001 cost= 5.774674465
Epoch: 0001 cost= 5.783594991
Epoch: 0001 cost= 5.791098959
Epoch: 0001 cost= 5.799115519
Epoch: 0001 cost= 5.806449034
Epoch: 0001 cost= 5.814601737
Epoch: 0001 cost= 5.821265795
Epoch: 0001 cost= 5.827086295
Epoch: 0001 cost= 5.833077600
Epoch: 0001 cost= 5.839360020
Epoch: 0001 cost= 5.847727233
Epoch: 0001 cost= 5.855888129
Epoch: 0001 cost= 5.862193859
Epoch: 0001 cost= 5.869057532
Epoch: 0001 cost= 5.879158278
Epoch: 0001 cost= 5.886835038
Epoch: 0001 cost= 5.894597409
Epoch: 000

Epoch: 0011 cost= 0.025158077
Epoch: 0011 cost= 0.026550527
Epoch: 0011 cost= 0.027553064
Epoch: 0011 cost= 0.028695901
Epoch: 0011 cost= 0.030364412
Epoch: 0011 cost= 0.032041565
Epoch: 0011 cost= 0.033256293
Epoch: 0011 cost= 0.034968206
Epoch: 0011 cost= 0.035922980
Epoch: 0011 cost= 0.036776413
Epoch: 0011 cost= 0.037808586
Epoch: 0011 cost= 0.038776215
Epoch: 0011 cost= 0.039746234
Epoch: 0011 cost= 0.040647792
Epoch: 0011 cost= 0.042047626
Epoch: 0011 cost= 0.043880824
Epoch: 0011 cost= 0.045070415
Epoch: 0011 cost= 0.046210007
Epoch: 0011 cost= 0.047282898
Epoch: 0011 cost= 0.048324545
Epoch: 0011 cost= 0.049429010
Epoch: 0011 cost= 0.050713913
Epoch: 0011 cost= 0.052136917
Epoch: 0011 cost= 0.053391378
Epoch: 0011 cost= 0.054687827
Epoch: 0011 cost= 0.056547430
Epoch: 0011 cost= 0.057709675
Epoch: 0011 cost= 0.059205715
Epoch: 0011 cost= 0.060416540
Epoch: 0011 cost= 0.061619095
Epoch: 0011 cost= 0.062742186
Epoch: 0011 cost= 0.064587631
Epoch: 0011 cost= 0.066082595
Epoch: 001

Epoch: 0011 cost= 0.357135258
Epoch: 0011 cost= 0.358602746
Epoch: 0011 cost= 0.359705660
Epoch: 0011 cost= 0.360815592
Epoch: 0011 cost= 0.362295664
Epoch: 0011 cost= 0.363785396
Epoch: 0011 cost= 0.364911526
Epoch: 0011 cost= 0.366073144
Epoch: 0011 cost= 0.367051772
Epoch: 0011 cost= 0.368589874
Epoch: 0011 cost= 0.369656112
Epoch: 0011 cost= 0.370688050
Epoch: 0011 cost= 0.371779777
Epoch: 0011 cost= 0.372797907
Epoch: 0011 cost= 0.373580018
Epoch: 0011 cost= 0.374403017
Epoch: 0011 cost= 0.375287413
Epoch: 0011 cost= 0.376687812
Epoch: 0011 cost= 0.377745440
Epoch: 0011 cost= 0.378633907
Epoch: 0011 cost= 0.379687895
Epoch: 0011 cost= 0.380959228
Epoch: 0011 cost= 0.382496848
Epoch: 0011 cost= 0.383516318
Epoch: 0011 cost= 0.384288127
Epoch: 0011 cost= 0.385293182
Epoch: 0011 cost= 0.386279398
Epoch: 0011 cost= 0.387505051
Epoch: 0011 cost= 0.388832901
Epoch: 0011 cost= 0.389768433
Epoch: 0011 cost= 0.390732338
Epoch: 0011 cost= 0.391747005
Epoch: 0011 cost= 0.392818724
Epoch: 001

Epoch: 0021 cost= 0.023819495
Epoch: 0021 cost= 0.024367998
Epoch: 0021 cost= 0.025155630
Epoch: 0021 cost= 0.025831059
Epoch: 0021 cost= 0.026618526
Epoch: 0021 cost= 0.027294486
Epoch: 0021 cost= 0.028052005
Epoch: 0021 cost= 0.028794871
Epoch: 0021 cost= 0.029479970
Epoch: 0021 cost= 0.029904485
Epoch: 0021 cost= 0.030423045
Epoch: 0021 cost= 0.031047041
Epoch: 0021 cost= 0.031701597
Epoch: 0021 cost= 0.032551626
Epoch: 0021 cost= 0.033011949
Epoch: 0021 cost= 0.033471250
Epoch: 0021 cost= 0.033885148
Epoch: 0021 cost= 0.034566073
Epoch: 0021 cost= 0.035165391
Epoch: 0021 cost= 0.035895442
Epoch: 0021 cost= 0.036287675
Epoch: 0021 cost= 0.036752482
Epoch: 0021 cost= 0.037251241
Epoch: 0021 cost= 0.037971092
Epoch: 0021 cost= 0.038784273
Epoch: 0021 cost= 0.039281053
Epoch: 0021 cost= 0.039724489
Epoch: 0021 cost= 0.040134778
Epoch: 0021 cost= 0.040768125
Epoch: 0021 cost= 0.041396821
Epoch: 0021 cost= 0.041946255
Epoch: 0021 cost= 0.042265606
Epoch: 0021 cost= 0.042649645
Epoch: 002

...

Epoch: 0071 cost= 0.092425169
Epoch: 0071 cost= 0.092568972
Epoch: 0071 cost= 0.092786193
Epoch: 0071 cost= 0.093222743
Epoch: 0071 cost= 0.093593126
Epoch: 0071 cost= 0.093799604
Epoch: 0071 cost= 0.094061474
Epoch: 0071 cost= 0.094297970
Epoch: 0071 cost= 0.094755810
Epoch: 0071 cost= 0.094970131
Epoch: 0071 cost= 0.095107350
Epoch: 0071 cost= 0.095569111
Epoch: 0071 cost= 0.095709763
Epoch: 0071 cost= 0.095996468
Epoch: 0071 cost= 0.096421688
Epoch: 0071 cost= 0.096590974
Epoch: 0071 cost= 0.096845561
Epoch: 0071 cost= 0.096997857
Epoch: 0071 cost= 0.097580079
Epoch: 0071 cost= 0.097913243
Epoch: 0071 cost= 0.098197554
Epoch: 0071 cost= 0.098407594
Epoch: 0071 cost= 0.098681841
Epoch: 0071 cost= 0.098790862
Epoch: 0071 cost= 0.099078808
Epoch: 0071 cost= 0.099297502
Epoch: 0071 cost= 0.099554323
Epoch: 0071 cost= 0.100136823
Epoch: 0071 cost= 0.100360303
Epoch: 0071 cost= 0.100583635
Epoch: 0071 cost= 0.100648795
Epoch: 0071 cost= 0.100891045
Epoch: 0071 cost= 0.101070284
Epoch: 007

Epoch: 0081 cost= 0.024110510
Epoch: 0081 cost= 0.024348310
Epoch: 0081 cost= 0.024732302
Epoch: 0081 cost= 0.025155247
Epoch: 0081 cost= 0.025395278
Epoch: 0081 cost= 0.025727102
Epoch: 0081 cost= 0.025856819
Epoch: 0081 cost= 0.025990230
Epoch: 0081 cost= 0.026193594
Epoch: 0081 cost= 0.026571352
Epoch: 0081 cost= 0.026796582
Epoch: 0081 cost= 0.026959152
Epoch: 0081 cost= 0.027243920
Epoch: 0081 cost= 0.027307813
Epoch: 0081 cost= 0.027651671
Epoch: 0081 cost= 0.027849105
Epoch: 0081 cost= 0.028156929
Epoch: 0081 cost= 0.028556811
Epoch: 0081 cost= 0.028725797
Epoch: 0081 cost= 0.028901679
Epoch: 0081 cost= 0.029083061
Epoch: 0081 cost= 0.029452870
Epoch: 0081 cost= 0.029758476
Epoch: 0081 cost= 0.029825900
Epoch: 0081 cost= 0.029952144
Epoch: 0081 cost= 0.030109153
Epoch: 0081 cost= 0.030244986
Epoch: 0081 cost= 0.030437083
Epoch: 0081 cost= 0.030544270
Epoch: 0081 cost= 0.030704439
Epoch: 0081 cost= 0.030943002
Epoch: 0081 cost= 0.031097647
Epoch: 0081 cost= 0.031484312
Epoch: 008

Epoch: 0081 cost= 0.095309442
Epoch: 0081 cost= 0.095589530
Epoch: 0081 cost= 0.095905177
Epoch: 0081 cost= 0.096027917
Epoch: 0081 cost= 0.096260045
Epoch: 0081 cost= 0.096599705
Epoch: 0081 cost= 0.096747934
Epoch: 0081 cost= 0.096916722
Epoch: 0081 cost= 0.097291458
Epoch: 0081 cost= 0.097443941
Epoch: 0081 cost= 0.097779985
Epoch: 0081 cost= 0.098032999
Epoch: 0081 cost= 0.098287066
Epoch: 0081 cost= 0.098443534
Epoch: 0081 cost= 0.098817579
Epoch: 0081 cost= 0.098989333
Epoch: 0081 cost= 0.099040910
Epoch: 0081 cost= 0.099297412
Epoch: 0081 cost= 0.099729563
Epoch: 0081 cost= 0.099979953
Epoch: 0081 cost= 0.100044957
Epoch: 0081 cost= 0.100101214
Epoch: 0081 cost= 0.100185247
Epoch: 0081 cost= 0.100509587
Epoch: 0081 cost= 0.100868857
Epoch: 0081 cost= 0.101087458
Epoch: 0081 cost= 0.101343232
Epoch: 0081 cost= 0.101518871
Epoch: 0081 cost= 0.101763750
Epoch: 0081 cost= 0.102054273
Epoch: 0081 cost= 0.102217121
Epoch: 0081 cost= 0.102338395
Epoch: 0081 cost= 0.102578588
Epoch: 008

Epoch: 0091 cost= 0.028872652
Epoch: 0091 cost= 0.029189292
Epoch: 0091 cost= 0.029406428
Epoch: 0091 cost= 0.029694234
Epoch: 0091 cost= 0.029809465
Epoch: 0091 cost= 0.030164798
Epoch: 0091 cost= 0.030439479
Epoch: 0091 cost= 0.030841617
Epoch: 0091 cost= 0.030966993
Epoch: 0091 cost= 0.031213067
Epoch: 0091 cost= 0.031318563
Epoch: 0091 cost= 0.031750256
Epoch: 0091 cost= 0.032019187
Epoch: 0091 cost= 0.032434850
Epoch: 0091 cost= 0.032653301
Epoch: 0091 cost= 0.032958301
Epoch: 0091 cost= 0.033156184
Epoch: 0091 cost= 0.033284385
Epoch: 0091 cost= 0.033606884
Epoch: 0091 cost= 0.033847250
Epoch: 0091 cost= 0.034064043
Epoch: 0091 cost= 0.034186304
Epoch: 0091 cost= 0.034365167
Epoch: 0091 cost= 0.034625734
Epoch: 0091 cost= 0.034874972
Epoch: 0091 cost= 0.035002573
Epoch: 0091 cost= 0.035166570
Epoch: 0091 cost= 0.035402257
Epoch: 0091 cost= 0.035548769
Epoch: 0091 cost= 0.035647123
Epoch: 0091 cost= 0.035746006
Epoch: 0091 cost= 0.036023470
Epoch: 0091 cost= 0.036403907
Epoch: 009

Epoch: 0091 cost= 0.095987218
Epoch: 0091 cost= 0.096218429
Epoch: 0091 cost= 0.096386928
Epoch: 0091 cost= 0.096602788
Epoch: 0091 cost= 0.096731788
Epoch: 0091 cost= 0.096972708
Epoch: 0091 cost= 0.097081071
Epoch: 0091 cost= 0.097627591
Epoch: 0091 cost= 0.097770971
Epoch: 0091 cost= 0.098039397
Epoch: 0091 cost= 0.098404325
Epoch: 0091 cost= 0.098630542
Epoch: 0091 cost= 0.098908964
Epoch: 0091 cost= 0.099104464
Epoch: 0091 cost= 0.099316784
Epoch: 0091 cost= 0.099617344
Epoch: 0091 cost= 0.099841960
Epoch: 0091 cost= 0.100021714
Epoch: 0091 cost= 0.100187546
Epoch: 0091 cost= 0.100466848
Epoch: 0091 cost= 0.100768086
Epoch: 0091 cost= 0.100848738
Epoch: 0091 cost= 0.101106102
Epoch: 0091 cost= 0.101511994
Epoch: 0091 cost= 0.101609729
Epoch: 0091 cost= 0.101896432
Epoch: 0091 cost= 0.102401373
Epoch: 0091 cost= 0.102561171
Epoch: 0091 cost= 0.102707769
Epoch: 0091 cost= 0.102862985
Epoch: 0091 cost= 0.102965792
Epoch: 0091 cost= 0.103315135
Epoch: 0091 cost= 0.103496572
Epoch: 009